In [1]:
import torch
from peft import PeftModel
from peft import prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
 
model_name_or_path = "/root/Mistral-7B-Instruct-v0.1"
lora_path = "/root/gritlm/gritlm/output/msmarco_train/checkpoint-28000"
output_path = "/root/gritlm/gritlm/output/mistral_sparse"
def apply_lora(model_name_or_path, output_path, lora_path):
    print(f"Loading the base model from {model_name_or_path}")
    base = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=True
    )
    # base = prepare_model_for_kbit_training(
    #     base, use_gradient_checkpointing=True
    # )
    base_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
    
    print(f"Loading the LoRA adapter from {lora_path}")
    from peft import get_peft_model, LoraConfig, TaskType
    # https://github.com/texttron/tevatron/blob/2e5d00ee21d5a7db0bd2ea1463c9150a572106d4/examples/repllama/repllama.py#L81
    # https://github.com/allenai/open-instruct/blob/9ebcb582cfc243a6dab75b4302fa432784db26c2/open_instruct/finetune.py#L478
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        r=16, 
        lora_alpha=64,
        lora_dropout=0.1,
        target_modules=["q_proj", "o_proj", "v_proj", "k_proj", "w1", "w2", "w3"]
    )
    model = get_peft_model(base, peft_config)
    # model.config.to_json_file("/root/gritlm/gritlm/output/msmarco_train/checkpoint-28000/adapter_config.json")
    # lora_model = model.load_adapter(lora_path, 'lora')
    lora_model = model.from_pretrained(
        model=base,
        model_id = lora_path,
        torch_dtype=torch.float16,
    )
 
    print("Applying the LoRA")
    # model = lora_model.merge_and_unload()
 
    print(f"Saving the target model to {output_path}")
    # model.save_pretrained(output_path)
    # base_tokenizer.save_pretrained(output_path)

/root/miniconda3/envs/grit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
apply_lora(model_name_or_path, output_path, lora_path)

Loading the base model from /root/Mistral-7B-Instruct-v0.1


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.49it/s]


Loading the LoRA adapter from /root/gritlm/gritlm/output/msmarco_train/checkpoint-28000


KeyError: 'peft_type'